In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import matplotlib as mpl
import requests
import re

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

mpl.rcParams['font.family'] = "Malgun Gothic"

In [3]:
url2 = 'https://lol.gamepedia.com/Special:RunQuery/PickBanHistory?PBH%5Bpage%5D=LCK+2021+Spring&PBH%5Btextonly%5D=Yes&pfRunQueryFormName=PickBanHistory'
ban_pick = pd.read_html(url2,encoding='utf-8')
ban_pick_df = pd.DataFrame(ban_pick[0])
ban_pick_df.columns = ban_pick_df.columns.droplevel()
ban_pick_df.shape

(215, 36)

In [4]:
ban_pick_df

,Phase,Blue,Red,Score,Winner,Patch,BB1,RB1,BB2,RB2,...,BR4,BR5,RR1,RR2,RR3,RR4,RR5,SB,MH,VOD
0,Week 10,Liiv SANDBOX,DRX,2 - 1,1,11.5,Seraphine,Gragas,Renekton,Hecarim,...,Mid,Top,Jungle,Bot,Top,Support,Mid,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://youtu.be/JwiT-V9vmBk?t=18143
1,Week 10,DRX,Liiv SANDBOX,1 - 1,1,11.5,Thresh,Seraphine,Azir,Udyr,...,Bot,Support,Top,Jungle,Bot,Mid,Support,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://youtu.be/JwiT-V9vmBk?t=14846
2,Week 10,Liiv SANDBOX,DRX,1 - 0,1,11.5,Thresh,Camille,Renekton,Hecarim,...,Mid,Support,Jungle,Bot,Top,Mid,Support,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://youtu.be/JwiT-V9vmBk?t=10860
3,Week 10,T1,Hanwha Life Esports,2 - 0,1,11.5,Lucian,Seraphine,Renekton,Hecarim,...,Bot,Support,Jungle,Top,Bot,Support,Mid,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://youtu.be/JwiT-V9vmBk?t=4829
4,Week 10,Hanwha Life Esports,T1,0 - 1,2,11.5,Gnar,Lucian,Udyr,Sion,...,Bot,Support,Mid,Bot,Support,Jungle,Top,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://youtu.be/JwiT-V9vmBk?t=1737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Week 1,T1,Hanwha Life Esports,2 - 1,1,11.1,Lucian,Pantheon,Akali,Aphelios,...,Support,Mid,Jungle,Bot,Top,Support,Mid,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://www.youtube.com/watch?v=J9FpxhAAqW0
211,Week 1,Hanwha Life Esports,T1,1 - 1,1,11.1,Lucian,Taliyah,Akali,Renekton,...,Support,Mid,Jungle,Mid,Bot,Top,Support,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://www.youtube.com/watch?v=WQhPkLRJugo
212,Week 1,T1,Hanwha Life Esports,1 - 0,1,11.1,Lucian,Pantheon,Azir,Taliyah,...,Mid,Top,Bot,Jungle,Support,Mid,Top,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://www.youtube.com/watch?v=6dk49hjLDTs
213,Week 1,KT Rolster,Gen.G,0 - 2,2,11.1,Gragas,Samira,Nidalee,Taliyah,...,Mid,Top,Bot,Jungle,Support,Mid,Top,SB,http://matchhistory.na.leagueoflegends.com/en/...,https://youtu.be/LXQ2hehVkw4


In [5]:
teams = ban_pick_df['Red'].unique()
teams = [k.replace(' ','+') for k in teams]
print(len(teams))

10


In [6]:
urls_list = []
for team in teams:
    url_new = 'https://lol.fandom.com/wiki/Special:RunQuery/PickBanHistory?pfRunQueryFormName=PickBanHistory&PBH%5Bpage%5D=LCK+2021+Spring&PBH%5Bteam%5D={0}&PBH%5Btextonly%5D%5Bis_checkbox%5D=true&PBH%5Btextonly%5D%5Bvalue%5D=1&wpRunQuery=Run+query&pf_free_text='
    urls = url_new.format(team)
    
    html2 = requests.get(urls).text
    soup2 = bs(html2,'html.parser')
    
    tmp_parsed2 = soup2.find_all('a',{'target' : '_blank',
                                      'rel' : 'nofollow noreferrer noopener',
                                      'class' : 'external free',
                                      'href' : re.compile('http://matchhistory')})
                                                  
    tmp_list = [k.text for k in tmp_parsed2]
    urls_list += tmp_list
urls_list = list(set(urls_list))

In [7]:
# basic link for selenium
link_sel = 'C:/Users/rsh15/Desktop/바탕/seunghuni/selenium'

In [13]:
browser = webdriver.Chrome(link_sel+'/chromedriver.exe')
browser.get(urls_list[0])
# browser.maximize_window()

In [14]:
html = browser.page_source
soup = bs(html,'html.parser')

In [15]:
if_signin = soup.find_all('h5',{'data-testid':'panel-title'})[0]
if_signin.text == 'Sign In'

True

In [17]:
browser.find_element_by_name('username').send_keys('dbtmdgns1502')
browser.find_element_by_name('password').send_keys('simon1304!')
browser.find_element_by_css_selector('body > div > div > div > div.grid.grid-direction__row.grid-page-web__content > div > div > button').click()

In [18]:
mh_html = bs(browser.page_source,'html.parser')
match_time = mh_html.find_all('div',{'id':'binding-698',
                              'class':'binding'})[0].text
match_time

'30:01'

In [ ]:
mh_html.find_all('div',{'class':'tower-kills'})


In [ ]:
obj_tags = ['tower-kills','inhibitor-kills','baron-kills','dragon-kills','rift-herald-kills']
tmp_dict = dict()
for tag in obj_tags:
    tmp_html = mh_html.find_all('div',{'class':tag})
    tmp_dict['blue_'+tag] = int(tmp_html[0].text.strip())
    tmp_dict['red_'+tag]  = int(tmp_html[1].text.strip())


In [ ]:
tmp_dict

In [ ]:
browser.find_element_by_id('tab-695').click()

In [ ]:
mh_table_html = bs(browser.page_source,'html.parser')
mh_table_html = mh_table_html.find_all('div',{'class':'champion-icon binding'})
mh_table_html